**Author:** Ashton Reimer

**Date:** 31 May 2021

### **PWM on the Arduino Nano**

Here I'm playing around with PWM on an Arduino Nano (Nano V3.0 ATmega328P Board). PWM is incredibly useful for a wide variety of projects, such as pure sine wave generators and charge controllers.


### **Generating a PWM signal**

The Arduino Nano has a bunch of pins on it:
<div>
<img src="images/nano.png" width="400">
</div>

Here's a description of the input/output capability of some of the pins (from https://store.arduino.cc/usa/arduino-nano):

Unsurprisingly, it's trivial to do PWM. Reading the Arduino documentation (https://www.arduino.cc/reference/en/language/functions/analog-io/analogwrite/), we just need to use the `analogWrite` function. Using this, I built a program that step through 0% to 100% to 0% duty cycle in ~1.96% increments (5/255).

<div>
<img src="images/pwm.gif" width="400">
</div>

Reading more documentation, for the Atmega 328, the PWM functionality is controlled by timers, so we should be able to change the frequency of the PWM by changing the timer used to generate it.


### **More Advanced: Changing Frequency**

Following the documentation (https://www.arduino.cc/en/Tutorial/SecretsOfArduinoPWM), we can change the frequency by changing timer configuration. We need to be careful not to mess with Timer 0, since it is used for the `delay` and `millis` functions. So we're going to mess with Timer 2 (which affects pins 11 and 3). 

This isn't quite as easy because we need to read the datasheet for the Atmega 328P and then decipher the timer control registers. After a bunch of reading and writing some things down:
 - the timer prescaler is selected by setting the first 3 bits of the TCCR2B register (pg 165)
 - we can change between "phase-correct" and "fast PWM" modes by setting bits in both the TCCR2B and TCCR2A registers (pgs 162, 164, and 165)
 
I ended up configuring the prescaler to 1 and using the "phase-correct" mode:
```
TCCR2A = 0b00000001
TCCR2B = 0b00000001
```
which run at a frequency of ~32 kHz. To use "Fast PWM":
```
TCCR2A = 0b00000011
```
which approximately doubles the frequency.